<a href="https://colab.research.google.com/github/ciepielajan/Geo_folium_map/blob/main/folium_draw_load_save_dual_geopandas_geojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium  # mapa
from folium.plugins import Draw  # rysownia


tiles = "My maps"
zoom_start = 13
location = [50.05,19.95]  # kraków


m = folium.Map(
    location=location, attr=tiles, zoom_start=zoom_start
)


folium.Circle(
    radius=100,
    location=location,
    popup="The Waterfront",
    color="crimson",
    fill=False,
).add_to(m)

draw = Draw(export=True,    # przycisk na mapie
            filename='data.geojson',)     # przycisk na mapie
draw.add_to(m)

m

In [ ]:
m.save("index.html")

In [ ]:
draw.save("index-draw.html")

`wczytanie pliku i naniesieni na mapę`

In [ ]:
!gdown --id "1sqKY43ZbZIAU87OCzaQF1Ab5x6nFKXfw"  # przykłądowy plik

Downloading...
From: https://drive.google.com/uc?id=1sqKY43ZbZIAU87OCzaQF1Ab5x6nFKXfw
To: /content/data.geojson
100% 1.38k/1.38k [00:00<00:00, 2.51MB/s]


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
import folium

m = folium.Map(location=[50.05,19.95],
               zoom_start=12,
          #  tiles='https://server.arcgisonline.com/arcgis/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{z}/{y}/{x}',
           attr='Własny tytuł mapy')


g = folium.GeoJson(
    "data.geojson",
).add_to(m)

m

`wczytanie pliku do dataframe`

In [ ]:
!pip -q install geopandas

     |████████████████████████████████| 1.0MB 6.2MB/s 
     |████████████████████████████████| 6.5MB 36.1MB/s 
     |████████████████████████████████| 14.8MB 218kB/s 


In [ ]:
import geopandas

In [ ]:
df = geopandas.read_file("data.geojson")
df

,geometry
0,"LINESTRING (19.80793 50.08570, 19.81230 50.060..."
1,POINT (19.78567 50.07650)
2,POINT (20.00061 50.05626)
3,POINT (20.08685 50.08046)
4,POINT (19.89042 50.03415)
5,"POLYGON ((19.91952 50.03521, 19.91952 50.07611..."
6,POINT (19.86547 50.06159)


In [ ]:
m = folium.Map([50.05,19.95], zoom_start=12,
               #tiles="cartodbpositron"
               )

folium.GeoJson(df).add_to(m)

m

In [ ]:
m.save("index.html")

`AUTO ZOOM`

In [ ]:
print(df.to_markdown())

|    | geometry                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#zamiana POLYGON  i LINESTRING na point żeby obliczyć najbardziej skrajne koordynaty
from shapely.geometry import Point

nodes = geopandas.GeoDataFrame(columns=["geometry"])

for i in df.index.to_list():
  if df['geometry'].iloc[i].geom_type=='Polygon':
    for j in df.iloc[i]["geometry"].exterior.coords:
      nodes = nodes.append({'geometry': Point(j) },ignore_index=True)

  elif df['geometry'].iloc[i].geom_type=='LineString':
    for j in df.iloc[i]["geometry"].coords:
      nodes = nodes.append({'geometry': Point(j) },ignore_index=True)

  elif df['geometry'].iloc[i].geom_type=='Point':
    nodes = nodes.append({'geometry': df['geometry'].iloc[i] },ignore_index=True)

#podgląd losowego indexu
nodes.sample()

,geometry
1,POINT (19.81230 50.06085)


In [ ]:
m = folium.Map(location=[nodes["geometry"].y.mean() , nodes["geometry"].x.mean()], 
               tiles="cartodbpositron"
               )

folium.GeoJson(df).add_to(m)


x_min = nodes["geometry"].x.min()
x_max = nodes["geometry"].x.max()
y_min = nodes["geometry"].y.min()
y_max = nodes["geometry"].y.max()

# szukanie lewego górnego rogu i dolnego prawego
m.fit_bounds([[y_max , x_min], [y_min , x_max]]) 

m

`DualMap`

In [ ]:
import folium.plugins

In [ ]:
m = folium.plugins.DualMap(location=(50.05,19.95), tiles=None, zoom_start=12)

folium.TileLayer("openstreetmap").add_to(m.m1)
folium.TileLayer("cartodbpositron").add_to(m.m2)

folium.GeoJson(df).add_to(m.m1)
folium.GeoJson(df).add_to(m.m2)

folium.LayerControl(collapsed=False).add_to(m)

m

In [ ]:
m.save('map.html')

`create Point from columns (Lat	Lon)  dataframe `

In [3]:
!pip -q install geopandas

     |████████████████████████████████| 1.0MB 11.6MB/s 
     |████████████████████████████████| 14.8MB 287kB/s 
     |████████████████████████████████| 6.5MB 55.7MB/s 


In [4]:
import geopandas
import pandas as pd

In [18]:
df = pd.DataFrame({'Lat': {0: 51.97430555559714,
  1: 51.998750000041596,
  2: 51.74958333337473,
  3: 51.64208333337464,
  4: 52.05097222226386,
  5: 51.804305555597,
  6: 51.80375000004144,
  7: 51.78208333337476,
  8: 51.67847222226357,
  9: 51.61902777781907},
 'Lon': {0: 22.4115277777957,
  1: 22.40986111112904,
  2: 22.40319444446236,
  3: 22.420416666684602,
  4: 22.45263888890685,
  5: 22.450694444462403,
  6: 22.45041666668462,
  7: 22.452083333351293,
  8: 22.472083333351307,
  9: 22.469027777795752},
 'Population': {0: 1.1497080299747129,
  1: 1.1497080299747129,
  2: 0.9836909145260389,
  3: 1.3843517013368698,
  4: 0.7502022877194049,
  5: 1.2838597540195849,
  6: 1.2838597540195849,
  7: 1.0699048508446118,
  8: 1.3843517013368698,
  9: 1.1275530278589474}}
  )

df[:2]

,Lat,Lon,Population
0,51.974306,22.411528,1.149708
1,51.998750,22.409861,1.149708


In [19]:
geodf = geopandas.GeoDataFrame(df)
geodf[:2]

,Lat,Lon,Population
0,51.974306,22.411528,1.149708
1,51.998750,22.409861,1.149708


In [14]:
from shapely.geometry import Point
def geo_point(x):
  return Point(x.Lon, x.Lat)

In [20]:
geodf["geometry"] = geodf.apply(geo_point, axis=1)
geodf[:2]

,Lat,Lon,Population,geometry
0,51.974306,22.411528,1.149708,POINT (22.41153 51.97431)
1,51.998750,22.409861,1.149708,POINT (22.40986 51.99875)


In [23]:
geodf.crs="epsg:4326"

In [21]:
import folium

In [24]:
m = folium.Map(location=[52 , 20], 
               tiles="cartodbpositron",
               zoom_start=7
               )

folium.GeoJson(
    geodf["geometry"]
    ).add_to(m)

m